In [1]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import glob

In [23]:
# Import data
df_map = pd.read_csv("20220514_mapping_processed.csv")
df_meta = pd.read_csv('../../figures/interactive_footprints/20231207_footprints_meta.csv')
df_regulonDB = pd.read_csv('../../figures/interactive_footprints/regulonDB_meta.csv')

df_fp = pd.read_csv('../../figures/interactive_footprints/footprints.csv')
df_fp = df_fp[(df_fp.replicate == 1)]

df_exshift = pd.read_csv('../../figures/interactive_footprints/exshifts.csv')
df_exshift = df_exshift[df_exshift.replicate == 1]

In [31]:
def plot_footprint(footprint, ax, promoter, growth_condition, d):

	max_signal = max(footprint)
	ax.set_ylim(top=max_signal*1.15)

	x = np.arange(-115+d, 45-d)
	#shiftcolors = [('#D56C55' if exshift > 0 else '#738FC1') for exshift in exshift_list]
	ax.bar(x, footprint, color='#738FC1', edgecolor=None, linewidth=0)
	ax.set_title(f'{growth_condition}', fontsize=8, loc='left')

	ax.tick_params(axis='x', labelsize=6)
	ax.tick_params(axis='y', labelsize=6)

def plot_exshift(exshift, ax_ex):
	vmax = np.max(np.abs(exshift))
	vmin = -vmax

	div_colors = sns.diverging_palette(258, 16, s=56, l=50, n=15, sep=5, center='light', as_cmap=True)
	
	hm = sns.heatmap(exshift, cmap=div_colors, ax=ax_ex, vmin=vmin, vmax=vmax, center=0)
	
	cbar = hm.collections[0].colorbar
	cbar.ax.set_position([ax_ex.get_position().x1 + 0.01, ax_ex.get_position().y0, 0.02, ax_ex.get_position().height])
	#cbar.set_label('Expression\nshift', rotation=90, labelpad=15, fontsize=14)
	cbar.ax.tick_params(labelsize=5) 
	
	ax_ex.set_yticklabels(['A', 'C', 'G', 'T'], rotation=360, fontsize=6)

	# Explicitly set the x-ticks to match your desired labels
	tick_positions = np.arange(15, 161, 20)  # Adjust the divisor to match your number of labels
	tick_labels = np.arange(-100, 41, 20)
	ax_ex.set_xticks(tick_positions)  # Set the positions for the x-ticks
	# Set the labels for the x-ticks and rotate them
	ax_ex.set_xticklabels(tick_labels, rotation=360, fontsize=6)  # Set the labels for the x-ticks and rotate them

In [32]:
def plot_regseq(promoter, d):
	# Filter data for the given promoter
	df_fp_promoter = df_fp[(df_fp['promoter'] == promoter) & (df_fp['d'] == d)]
	df_exshift_promoter = df_exshift[df_exshift['promoter'] == promoter]
	
	growth_conditions = np.array(sorted(df_fp_promoter['growth_condition'].unique()))

	fp_list = []
	exshift_list = []

	for gc in growth_conditions:
		df_fp_gc = df_fp_promoter[df_fp_promoter['growth_condition'] == gc]
		df_exshift_gc = df_exshift_promoter[df_exshift_promoter['growth_condition'] == gc]

		footprint = df_fp_gc['mut_info'].values
		exshift = np.zeros((4, 160))
		for i in range(4):
			exshift[i, :] = df_exshift_gc[df_exshift_gc['base'] == (i+1)]['expression_shift'].values

		fp_list.append(footprint)
		exshift_list.append(exshift)
		
	for pdf_index in range(3):
		totnum = 10 if pdf_index < 2 else 11

		fig, axes = plt.subplots(totnum, 2, figsize=(8.5, 12))
		fig.text(0.0, 1.0, 'Promoter: {}'.format(promoter), va='center', ha='left', fontsize=12)

		fig.text(0.00, 0.5, 'Information (bits)', va='center', ha='center', rotation='vertical', fontsize=10)
		fig.text(0.98, 0.5, 'Expression shift', va='center', ha='center', rotation='vertical', fontsize=10)
		
		for i in range(totnum):
			ax_fp = axes[i][0]
			ax_ex = axes[i][1]
			footprint = fp_list[pdf_index * 10 + i]
			exshift = exshift_list[pdf_index * 10 + i]
			growth_condition = growth_conditions[pdf_index * 10 + i]

			if i == 0:
				plot_footprint(footprint, ax_fp, promoter, growth_condition, d)
				plot_exshift(exshift, ax_ex)
			else:
				plot_footprint(footprint, ax_fp, promoter, growth_condition, d)
				plot_exshift(exshift, ax_ex)

			if i == totnum - 1:
				ax_fp.set_xlabel('Position relative to TSS', fontsize=9)
				ax_ex.set_xlabel('Position Relative to TSS', fontsize=9)
		
		plt.tight_layout()
		plt.savefig('./footprints/{}_{}.pdf'.format(promoter, pdf_index), bbox_inches='tight')
		plt.close()

In [30]:
promoters = list(df_fp['promoter'].unique())

for promoter in ['mglBp']:#promoters:
	plot_regseq(promoter, 0)

In [7]:
import os
import PyPDF2

def get_promoter_pdfs(promoter, directory='./all_footprints'):
    pdf_files = []
    for filename in sorted(os.listdir(directory)):
        if filename.startswith(promoter) and filename.endswith('.pdf'):
            pdf_files.append(os.path.join(directory, filename))
    return pdf_files

def merge_pdfs(pdf_list, output_path):
    pdf_merger = PyPDF2.PdfMerger()
    
    for pdf in pdf_list:
        pdf_merger.append(pdf)
    
    with open(output_path, 'wb') as output_file:
        pdf_merger.write(output_file)

def merge_all_promoter_pdfs(promoters, input_dir='./all_footprints', output_dir='./merged_pdfs'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for promoter in promoters:
        pdf_files = get_promoter_pdfs(promoter, input_dir)
        if pdf_files:
            output_pdf = os.path.join(output_dir, '{}.pdf'.format(promoter))
            merge_pdfs(pdf_files, output_pdf)
            print(f"Merged PDF for promoter {promoter} saved as {output_pdf}")
        else:
            print(f"No PDF files found for promoter {promoter}")

# Example DataFrame (df_fp) to get the list of promoters
# df_fp = pd.read_csv('footprint_data.csv')
promoters = list(df_fp['promoter'].unique())

merge_all_promoter_pdfs(promoters)

Merged PDF for promoter TSS_1414_storz_regulondb saved as ./merged_pdfs/TSS_1414_storz_regulondb.pdf
Merged PDF for promoter TSS_18061_regulondb saved as ./merged_pdfs/TSS_18061_regulondb.pdf
Merged PDF for promoter TSS_4654_wanner_regulondb saved as ./merged_pdfs/TSS_4654_wanner_regulondb.pdf
Merged PDF for promoter TSS_4655_regulondb saved as ./merged_pdfs/TSS_4655_regulondb.pdf
Merged PDF for promoter TSS_6311_regulondb saved as ./merged_pdfs/TSS_6311_regulondb.pdf
Merged PDF for promoter aceBp saved as ./merged_pdfs/aceBp.pdf
Merged PDF for promoter acrAp saved as ./merged_pdfs/acrAp.pdf
Merged PDF for promoter acrBp saved as ./merged_pdfs/acrBp.pdf
Merged PDF for promoter acrZp saved as ./merged_pdfs/acrZp.pdf
Merged PDF for promoter araBp saved as ./merged_pdfs/araBp.pdf
Merged PDF for promoter araCp saved as ./merged_pdfs/araCp.pdf
Merged PDF for promoter blrp saved as ./merged_pdfs/blrp.pdf
Merged PDF for promoter cpxRp2 saved as ./merged_pdfs/cpxRp2.pdf
Merged PDF for promoter